In [10]:
import pandas as pd
from glob import glob
from pathlib import Path


In [ ]:
work_dfs = {}
for file in glob("./cultivos/*.csv"):
    df = pd.read_csv(file)
    filename= Path(file).stem.lower()
    work_dfs[filename] = df
    
work_dfs.keys()

dict_keys(['barley', 'sweet_potato', 'oilseeds_total', 'wheat', 'banana', 'tapioca'])

In [15]:
work_dfs["barley"][work_dfs["barley"]["Crop_Year"] == 1997]

,Annual_Rainfall,Pesticide,Fertilizer,Crop_Year,Yield
0,1852.9,1895.96,582059.72,1997,1.680000
11,1303.7,13279.78,4076892.46,1997,1.040000
12,1189.1,28272.00,8679504.00,1997,1.347097
13,690.1,11470.00,3521290.00,1997,2.995455
14,473.9,104481.47,32075811.29,1997,2.046329
218,661.2,13020.00,3997140.00,1997,2.596667
273,2804.1,406.10,124672.70,1997,1.155000
